In [47]:
# Import section
import cv2
import numpy as np
import requests
import time
import json
import pandas as pd
import sqlalchemy as sa
import pyodbc
import time
from paralel_send import multi_send
from os import getcwd

import argparse
import general_utils


In [48]:
# Info about the params file
p = 'params/campaigns'
camp_file = general_utils.get_conf_file_of_camp(p, 4)
print("[INFO] Cargando archivo de parámetros: {}".format(camp_file))
dni_area = general_utils.get_param_from_file(camp_file, 'dni_search_area')
traslation_dict = general_utils.get_param_from_file(camp_file, 'traslation_dict')
lines_dict = general_utils.get_param_from_file(camp_file, 'lines_dict')

# Folder params
WORK_DIRECTORY = getcwd()
ARRAYS_PATH = WORK_DIRECTORY + '\\areas_result\\'
TMP_PATH = WORK_DIRECTORY + '\\temps\\'
LOCAL_PATH = WORK_DIRECTORY + '\\local_result\\'
TMP_FAILED_PATH = TMP_PATH + '\\reproc\\'


[INFO] Cargando archivo de parámetros: params/marzo_compras


In [51]:
# Loading muestra numpy arrays

areas_r = pd.read_csv(ARRAYS_PATH + 'areas.csv')
areas_r = areas_r.fillna('')
print('Total cupones area: {}'.format(areas_r.path.values.size))


Total cupones area: 40000


In [52]:
# Moving temp files into reproc
import os
import shutil

filenames = [arch.name for arch in os.scandir(TMP_PATH) if arch.is_file()]

for f in filenames:
    shutil.move(TMP_PATH+f, TMP_FAILED_PATH+f)

In [54]:
# Getting all the processed files

filenames = [arch.name for arch in os.scandir(TMP_FAILED_PATH) if arch.is_file()]
files = []
jsons = []

for f in filenames:
    if 'n_files' in f:
        file = np.load(TMP_FAILED_PATH + f, allow_pickle=True)
        files.append(file)
    elif 'n_jsons' in f:
        azu = np.load(TMP_FAILED_PATH + f, allow_pickle=True)
        jsons.append(azu)
    else:
        print('->sin caso')

azure_files = np.concatenate(files[:], axis=0)
azure_jsons = np.concatenate(jsons[:], axis=0)
print('Resultado de azure: {}  {}'.format(azure_files.size, azure_jsons.size))

Resultado de azure: 25804  25804


In [55]:
14196 + 25804

40000

In [56]:
# Merging the areas result with the azure result
proc = pd.DataFrame({'path': azure_files, 'azure_json': azure_jsons})
proc = pd.merge(areas_r, proc, how='left', on='path')
print('Total cupones merge: {}'.format(proc.path.values.size))

Total cupones merge: 40000


In [58]:
proc[proc['azure_json'].isnull()]['path'].values.tolist()

['E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00303.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00304.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00306.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00307.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00308.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00309.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00311.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompras_AV_B4_20190709094729_00312.jpg',
 'E:\\git\\cupones_wong\\data\\escaneos\\marzo_compras_2018\\04\\B4\\MarzoCompra